In [ ]:
# Normalized schema (silver level medallion) modeling and export
import pandas as pd
from pathlib import Path

filepath = Path("../05_school/640/data/nationaldatabaseofchildcareprices.csv")
# Load data
df = pd.read_csv(filepath)

# --- 1. Geography Dimension ---
geography_cols = ['State_Name', 'State_Abbreviation', 'County_Name', 'County_FIPS_Code']
geography_dim = df[geography_cols].drop_duplicates().reset_index(drop=True)
geography_dim['Geography_Key'] = geography_dim.index + 1

# --- 2. Year Dimension ---
year_dim = df[['StudyYear']].drop_duplicates().reset_index(drop=True)
year_dim['Year_Key'] = year_dim.index + 1

# --- 3. Age Group Dimension ---
age_groups = [
    ('Bto5', 'Birth to 5 months'), ('6to11', '6 to 11 months'), ('12to17', '12 to 17 months'),
    ('18to23', '18 to 23 months'), ('24to29', '24 to 29 months'), ('30to35', '30 to 35 months'),
    ('36to41', '36 to 41 months'), ('42to47', '42 to 47 months'), ('48to53', '48 to 53 months'),
    ('54toSA', '54 months to school age'), ('SA', 'School age'), 
    ('Infant', 'Infant'), ('Toddler', 'Toddler'), ('Preschool', 'Preschool')
]
age_group_dim = pd.DataFrame(age_groups, columns=['Age_Group_Code', 'Age_Group_Description'])
age_group_dim['Age_Group_Key'] = age_group_dim.index + 1

# --- 4. Demographics Dimension ---
demographic_cols = [
    'TotalPop', 'OneRace', 'OneRace_W', 'OneRace_B', 'OneRace_I', 'OneRace_A', 'OneRace_H', 'OneRace_Other',
    'TwoRaces', 'Hispanic', 'iTotalPop', 'iOneRace', 'iOneRace_W', 'iOneRace_B', 'iOneRace_I', 'iOneRace_A',
    'iOneRace_H', 'iOneRace_Other', 'iTwoRaces', 'iHispanic'
]
demographics_dim = df[demographic_cols].drop_duplicates().reset_index(drop=True)
demographics_dim['Demographics_Key'] = demographics_dim.index + 1

# --- 5. Household Characteristics Dimension ---
household_cols = [
    'Households', 'H_Under6_BothWork', 'H_Under6_FWork', 'H_Under6_MWork', 'H_Under6_SingleM',
    'H_6to17_BothWork', 'H_6to17_Fwork', 'H_6to17_Mwork', 'H_6to17_SingleM',
    'iHouseholds', 'iH_Under6_BothWork', 'iH_Under6_FWork', 'iH_Under6_MWork', 'iH_Under6_SingleM',
    'iH_6to17_BothWork', 'iH_6to17_Fwork', 'iH_6to17_Mwork', 'iH_6to17_SingleM'
]
household_dim = df[household_cols].drop_duplicates().reset_index(drop=True)
household_dim['Household_Key'] = household_dim.index + 1

# --- 6. Labor Force Dimension ---
labor_force_cols = [
    'UNR_16', 'FUNR_16', 'MUNR_16', 'UNR_20to64', 'FUNR_20to64', 'MUNR_20to64',
    'FLFPR_20to64', 'FLFPR_20to64_Under6', 'FLFPR_20to64_6to17', 'FLFPR_20to64_Under6_6to17', 'MLFPR_20to64',
    'PR_F', 'PR_P', 'EMP_M', 'MEMP_M', 'FEMP_M', 'EMP_Service', 'MEMP_Service', 'FEMP_Service',
    'EMP_Sales', 'MEMP_Sales', 'FEMP_Sales', 'EMP_N', 'MEMP_N', 'FEMP_N', 'EMP_P', 'MEMP_P', 'FEMP_P',
    'iUNR_16', 'iFUNR_16', 'iMUNR_16', 'iUNR_20to64', 'iFUNR_20to64', 'iMUNR_20to64',
    'iFLFPR_20to64', 'iFLFPR_20to64_Under6', 'iFLFPR_20to64_6to17', 'iFLFPR_20to64_Under6_6to17', 'iMLFPR_20to64',
    'iPR_F', 'iPR_P', 'iEMP_M', 'iMEMP_M', 'iFEMP_M', 'iEMP_Service', 'iMEMP_Service', 'iFEMP_Service',
    'iEMP_Sales', 'iMEMP_Sales', 'iFEMP_Sales', 'iEMP_N', 'iMEMP_N', 'iFEMP_N', 'iEMP_P', 'iMEMP_P', 'iFEMP_P'
]
labor_force_dim = df[labor_force_cols].drop_duplicates().reset_index(drop=True)
labor_force_dim['Labor_Force_Key'] = labor_force_dim.index + 1

# --- 7. Income Dimension ---
income_cols = [
    'MHI', 'ME', 'FME', 'MME', 'MHI_2018', 'ME_2018', 'FME_2018', 'MME_2018',
    'iMHI', 'iME', 'iFME', 'iMME', 'iMHI_2018', 'iME_2018', 'iFME_2018', 'iMME_2018'
]
income_dim = df[income_cols].drop_duplicates().reset_index(drop=True)
income_dim['Income_Key'] = income_dim.index + 1

# --- 8. Fact Table: Childcare Prices ---
provider_types = {
    'MC': 'Center-Based',
    'MFCC': 'Family Child Care',
    '_75C': 'Center-Based',
    '_75FCC': 'Family Child Care'
}
price_types = {
    'MC': 'Median',
    'MFCC': 'Median',
    '_75C': '75th Percentile',
    '_75FCC': '75th Percentile'
}
price_columns = [
    'MCBto5', 'MC6to11', 'MC12to17', 'MC18to23', 'MC24to29', 'MC30to35', 'MC36to41', 'MC42to47', 'MC48to53', 'MC54toSA', 'MCSA',
    'MFCCBto5', 'MFCC6to11', 'MFCC12to17', 'MFCC18to23', 'MFCC24to29', 'MFCC30to35', 'MFCC36to41', 'MFCC42to47', 'MFCC48to53', 'MFCC54toSA', 'MFCCSA',
    '_75CBto5', '_75C6to11', '_75C12to17', '_75C18to23', '_75C24to29', '_75C30to35', '_75C36to41', '_75C42to47', '_75C48to53', '_75C54toSA', '_75CSA',
    '_75FCCBto5', '_75FCC6to11', '_75FCC12to17', '_75FCC18to23', '_75FCC24to29', '_75FCC30to35', '_75FCC36to41', '_75FCC42to47', '_75FCC48to53', '_75FCC54toSA', '_75FCCSA'
]
id_vars = geography_cols + ['StudyYear'] + demographic_cols + household_cols + labor_force_cols + income_cols
price_long = df[id_vars + price_columns].melt(
    id_vars=id_vars, 
    value_vars=price_columns, 
    var_name='Price_Column', 
    value_name='Price'
)

def parse_price_column(col):
    for prefix in provider_types:
        if col.startswith(prefix):
            provider = provider_types[prefix]
            price_type = price_types[prefix]
            age_group = col.replace(prefix, '')
            return provider, price_type, age_group
    return None, None, None

price_long[['Provider_Type', 'Price_Type', 'Age_Group_Code']] = price_long['Price_Column'].apply(
    lambda x: pd.Series(parse_price_column(x))
)

# Merge dimension keys
fact = price_long.merge(geography_dim, on=geography_cols, how='left')
fact = fact.merge(year_dim, on='StudyYear', how='left')
fact = fact.merge(age_group_dim, on='Age_Group_Code', how='left')
fact = fact.merge(demographics_dim, on=demographic_cols, how='left')
fact = fact.merge(household_dim, on=household_cols, how='left')
fact = fact.merge(labor_force_dim, on=labor_force_cols, how='left')
fact = fact.merge(income_dim, on=income_cols, how='left')

fact_table = fact[[
    'Geography_Key', 'Year_Key', 'Age_Group_Key', 'Demographics_Key', 'Household_Key',
    'Labor_Force_Key', 'Income_Key', 'Provider_Type', 'Price_Type', 'Price'
]]

# --- 9. Export to CSV ---
geography_dim.to_csv('geography_dim.csv', index=False)
year_dim.to_csv('year_dim.csv', index=False)
age_group_dim.to_csv('age_group_dim.csv', index=False)
demographics_dim.to_csv('demographics_dim.csv', index=False)
household_dim.to_csv('household_dim.csv', index=False)
labor_force_dim.to_csv('labor_force_dim.csv', index=False)
income_dim.to_csv('income_dim.csv', index=False)
fact_table.to_csv('childcare_price_fact.csv', index=False)

print("Exported: geography_dim.csv, year_dim.csv, age_group_dim.csv, demographics_dim.csv, household_dim.csv, labor_force_dim.csv, income_dim.csv, childcare_price_fact.csv")